In [ ]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver
import networkx as nx
import torch
from torch import DoubleTensor as Tensor

In [ ]:
base_input = '../../input_files/ap2_MADD.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

In [ ]:
print(rn.rxn_class)
print(rn.mon_rxn_map)
print(rn.dG_map)

In [ ]:
uid_dict = {}
react_dict = {}
sys.path.append("../../")
nodes_list = []
node_dict = {}
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    print(n,"--",gtostr(rn.network.nodes[n]['struct']))
    nodes_list.append(gtostr(rn.network.nodes[n]['struct']))
    node_dict[gtostr(rn.network.nodes[n]['struct'])] = n
    for r_set in rn.get_reactant_sets(n):
        r_tup = tuple(list(r_set)+[n])
#         print(r_tup)
        data = rn.network.get_edge_data(r_tup[0], n)
        reaction_id = data['uid']
        react_dict[r_tup]=reaction_id
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = ("".join(r1),"".join(r2))
#         print(reactants)
        uid_dict[(n,k)] = uid
#         react_dict[reactants] = uid

print(uid_dict)
print(react_dict)
print(rn._rxn_count)

In [ ]:
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
#new_kon = [0.2244, 0.2244, 0.2255, 1.6039, 1.6039, 1.6040] # M-1 s-1   
#new_kon = [0.0125, 0.1947, 0.0125, 3.6241, 3.4263, 3.4258] #Max yield Simruntime = 10sec
# new_kon = new_kon + Tensor([1.]*np.array(1e0))


"""
All rxns ; Capped
"""
#C5
# new_kon = new_kon + Tensor([9.940385, 9.913335, 9.92041, 9.918401, 9.915438, 0.160003, 0.193526, 9.942909, 0.191574, 0.156381, 0.122522, 9.948579, 0.1848, 0.159817, 0.172933, 9.91892, 8.206506, 9.908399, 9.912474, 0.10773, 0.186356, 0.154725, 0.163791, 9.944109, 9.929127])

#C3
# new_kon = new_kon + Tensor([4.382366, 9.925588, 9.931162, 9.932764, 9.925204, 0.192061, 0.179026, 4.398933, 0.16639, 0.182279, 0.184085, 6.736659, 0.169466, 0.175913, 0.193371, 6.526176, 1.145394, 5.861408, 5.895674, 0.139223, 0.147791, 0.193958, 0.195403, 6.924301, 6.704808])


"""
MADD Growth Model ; Capped
"""
# new_kon = new_kon + Tensor([1.518357, 1.854395, 0.145259, 2.020679, 0.148764, 9.903206, 8.532263, 0.154239, 9.905051, 8.566359, 8.565406, 9.900933, 6.586582, 6.64002, 6.671817, 8.424373, 9.905084, 8.511572, 8.501119, 8.520661, 9.905183, 9.911341])

#C5
new_kon = new_kon + Tensor([0.103692, 0.104735, 0.118055, 0.105349, 0.120976, 1.109127, 1.195351, 0.123504, 1.101268, 1.186702, 1.18933, 1.565666, 1.277947, 1.277865, 1.28311, 1.393554, 0.979466, 1.058906, 1.059051, 1.199964, 1.56619, 1.46319])
update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))

In [ ]:
vec_rn = VectorizedRxnNet(rn, dev='cpu')
vec_rn.reset()
# print(vec_rn.kon)
runtime =1
c_scale=1
sim = VecSim(vec_rn, runtime, device='cpu',rate_step=True)
y = sim.simulate(conc_scale=c_scale)

In [ ]:
print(vec_rn.copies_vec)
print(y[1])
print(new_kon.shape)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
fig, ax = plt.subplots()
nodes_list = ['A','B','M','S','ABMS']
# nodes_list_repeat = ['A','B','S','AB','AS','AA','BS','ABS','AAB','AAS','AABS']
sim.plot_observable(nodes_list, ax=ax,legend=False,seed=101)
ax.set_title("runtime: " + str(runtime) + " seconds")
handles,labels = ax.get_legend_handles_labels()
ax.set_xscale("log")
fig.legend(handles,nodes_list,loc='center right',fancybox=True,ncol=3,fontsize='small',markerscale=1.0)
ax.grid(which="major",axis="both")

In [ ]:
# for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
#     print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
import networkx as nx
labels = nx.get_node_attributes(rn.network, 'struct')
labels = {key:gtostr(labels[key]) for key in labels.keys()}
# nx.draw_networkx(rn.network,labels=labels,node_size=1000)

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
sl_pos = nx.shell_layout(rn.network)
nx.draw_networkx(rn.network,pos=sl_pos,labels=labels,node_size=1000,node_color='indianred',ax=ax,edgelist=[])
# new_kon = 0.5*np.array([0.8179, 0.8172, 0.8174, 0.8174, 0.8218, 0.0296, 0.0295, 4.6052, 0.8188,
#         0.8188, 7.7299, 5.4280, 0.8199, 0.8183, 7.1332, 4.9835, 4.9838, 0.0307,
#         0.0315, 5.2260, 5.2278])

if vec_rn.rxn_coupling:
    label_kon = 0.5*sim.coupled_kon.detach().numpy()
else:
    label_kon = 0.5*vec_rn.kon.detach().numpy()
    
sample_colors = ['black','black','black','black','olivedrab','orchid','darkorange','teal','']
edge_widths = []
edge_colors= []
for edge,uid in uid_dict.items():
    edge_widths.append(label_kon[uid])
    #edge_colors.append(sample_colors[uid])
nx.draw_networkx_edges(rn.network,pos=sl_pos,width=edge_widths,label=edge_widths,arrowsize=10,min_target_margin=15.0)


In [ ]:
uid_dict = {}
uid_reactants = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'uid':uid,'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        uid_reactants[uid]=reactants
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
def get_all_paths(species,node_dict):
    sp_node = node_dict[species]
#     print("Species: ",species)
    sub_sp_paths = []
#     for mon in species:
        
    for path in nx.all_simple_edge_paths(rn.network,node_dict[species[0]],sp_node):
        rate_routes = []
        flag=False
        for edges in path:
            data = rn.network.get_edge_data(edges[0],edges[1])
            if flag:
                for route in rate_routes:
                    route.append(data['uid'])
            else:    
                rate_routes.append(data['uid'])
            
            r1 = set(gtostr(rn.network.nodes[edges[0]]['struct']))
            p = set(gtostr(rn.network.nodes[edges[1]]['struct']))
            r2 = p-r1
            
            r2_name = "".join(sorted(list(r2)))
#             print(r1,r2_name,len(r2),data['uid'],flag)
            if len(r2_name)>1:
                extra_paths = get_all_paths(r2_name,node_dict)
#                 print("Extra Paths: ",extra_paths)
                
                if extra_paths:
                    prev_path = rate_routes
                    rate_routes = []
                    for ex in extra_paths:
                        new_path = prev_path+ex
                        rate_routes.append(new_path)
                        flag=True
#                         if new_path not in sub_sp_paths:
#                             flag=True
#                             sub_sp_paths.append(new_path)

            
        if (rate_routes not in sub_sp_paths) and not flag:
#             print("Final rate routes: ",rate_routes)
            sub_sp_paths.append(rate_routes)
            flag=False
        elif flag:
            sub_sp_paths = sub_sp_paths+rate_routes

    
#     print("Sub PATHS: ",sub_sp_paths)
    return(sub_sp_paths)

In [ ]:
last_node = list(rn.network.nodes())[-1]
monomer_rxns = {}

n_paths=0
for path in nx.all_simple_paths(rn.network,0,last_node):
    print(path)
    n_paths+=1

master_paths = []

for mn_nodes in [0]:
    for path in nx.all_simple_edge_paths(rn.network,mn_nodes,last_node):
        #     print(path)
            rate_routes = []
            flag=False
            for edges in path:
                data = rn.network.get_edge_data(edges[0],edges[1])

                if flag:
#                     print("Rate Routes: ",rate_routes)
                    for route in rate_routes:
                        route.append(data['uid'])
    #                 flag=False
                else:

                    rate_routes.append(data['uid'])
                r1 = set(gtostr(rn.network.nodes[edges[0]]['struct']))
                p = set(gtostr(rn.network.nodes[edges[1]]['struct']))
                r2 = p-r1
#                 print(r2)

                if len(r2)>1:
                    extra_paths = get_all_paths("".join(sorted(list(r2))),node_dict)

                    if extra_paths:
                        prev_path = rate_routes
                        rate_routes = []
                        for ex in extra_paths:
                            new_path = prev_path+ex
                            rate_routes.append(new_path)
                            flag=True

            if (rate_routes not in master_paths) and not flag:
    #             print("Final Rate routes: ",rate_routes)
                master_paths.append(rate_routes)
            elif flag:
                master_paths=master_paths+rate_routes

for node in rn.network.nodes():
    
    if not rn.network.in_edges(node):
        for path in nx.all_simple_edge_paths(rn.network,node,last_node):
            data01 = rn.network.get_edge_data(path[0][0],path[0][1])
            if node not in monomer_rxns.keys():
                monomer_rxns[node] = [data01['uid']]
            else:
                monomer_rxns[node].append(data01['uid'])

print(len(list(set(monomer_rxns[0]))))
print(n_paths)

In [ ]:
corrected_master_path = []
for path in master_paths:
    mod_path = []
    if type(path[0])==type([1,2]):
        mod_path=path[0]
        for r in range(1,len(path)):
            mod_path.append(path[r])
        
        if sorted(mod_path) not in corrected_master_path:
            corrected_master_path.append(sorted(mod_path))
    else:
        if sorted(path) not in corrected_master_path:
            corrected_master_path.append(sorted(path))
        
print(len(corrected_master_path))


print("All reaction pathways")

path_type = {'MADD':[],'DD':[]}
for path in corrected_master_path:
    pt_name = ''
    for rid in path:
        r1 = "".join(list(uid_reactants[rid][0]))
        r2 = "".join(list(uid_reactants[rid][1]))
        print("{:^4s} + {:^4s}".format(r1,r2),end=' | ')
        
        if len(r1)==2 and len(r2)==2:
            pt_name='DD'
            special_id=rid
        elif (len(r1)==1 and len(r2)==2) or (len(r2)==1 and len(r1)==2):
            pt_name='MADD'
            special_id=rid
#         print("{:^6d}".format(rid),end=' | ')
    print(path)
    path_type[pt_name].append(rid)

In [ ]:
madd_path_pot = np.zeros((len(path_type['MADD']),1))
dd_path_pot = np.zeros((len(path_type['DD']),1))


time_steps_02 = sim.steps
time_steps_02.append(sim.steps[-1])
steps_array = np.array(time_steps_02)
mask_step = (steps_array<=y[1][2].item())
# mask_step = (steps_array<=10)
# mask_step.append(False)

rate_step_array = np.zeros((len(sim.rate_step_array),2*rn._rxn_count))

print(rate_step_array.shape)
for i in range(len(sim.rate_step_array)):
    rate_step_array[i,:] = np.array(sim.rate_step_array[i].detach().numpy())

for i in range(len(path_type['MADD'])):
    curr_path_contri = rate_step_array[:,path_type['MADD'][i]]
    rev_indx = rn._rxn_count + path_type['MADD'][i]
    rev_path_contri = rate_step_array[:,rev_indx]
    
    net_path_contri = curr_path_contri + rev_path_contri
    madd_path_pot[i] =  np.sum(net_path_contri)*c_scale

for i in range(len(path_type['DD'])):
    curr_path_contri = rate_step_array[:,path_type['DD'][i]]
    rev_indx = rn._rxn_count + 1 + path_type['DD'][i]
    rev_path_contri = rate_step_array[:,rev_indx]
    
    net_path_contri = curr_path_contri + rev_path_contri
    dd_path_pot[i] =  np.sum(net_path_contri)*c_scale
#     print(dd_path_pot[i])
    


In [ ]:
# print(mask_step.shape,len(sim.rate_step_array),len(path_type['MADD'][i]))
print(path_type['MADD'][7])

In [ ]:
fig,ax = plt.subplots()
ax.plot(madd_path_pot,linestyle='',marker='o')
ax.plot(dd_path_pot,linestyle='',marker='x')
# ax.plot(dt_path_pot,linestyle='',marker='s')

In [ ]:
fig,ax=plt.subplots()
ax.plot(rate_step_array[:,path_type['DD'][0]])
ax.plot(rate_step_array[:,path_type['DD'][1]])
ax.plot(rate_step_array[:,path_type['DD'][2]])
print(path_type['DD'])